# Libraries

In [1]:
%pip install -q -U datasets transformers accelerate sentencepiece

In [2]:
import os
import random
import numpy as np
import torch
import transformers
from pprint import pprint
from datasets import load_dataset
from transformers import AutoModelForSequenceClassification, AutoTokenizer, DataCollatorWithPadding, Trainer, TrainingArguments
from sklearn.metrics import f1_score

# Config

In [4]:
seed = 42
lang = 'sun'
hf_model_id = 'w11wo/sundanese-bert-base-emotion-classifier'
hf_data_id = 'alxxtexxr/SemEval2025-Task11-Dataset'
hf_data_config = 'track_a_sun_70_15_15_stratify_v2'

In [5]:
def set_seed(seed):
    # Set random seed for NumPy
    np.random.seed(seed)

    # Set random seed for Torch
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # if using multi-GPU
    torch.backends.cudnn.deterministic = True  # Ensures deterministic results
    torch.backends.cudnn.benchmark = False  # Avoids non-deterministic algorithms

    # Set random seed for Transformers
    transformers.set_seed(seed)

    # Optionally set random seed for sklearn and Python's own random module
    random.seed(seed)

    # Set random seed for os
    os.environ['PYTHONHASHSEED'] = str(seed)

    print(f"Random seed set to: {seed}")

set_seed(seed)

Random seed set to: 42


# Data

## Load Data

In [8]:
datasets = load_dataset(hf_data_id, hf_data_config)

cols = list(datasets['train'].features)
emotion_cols = [col for col in cols if col not in ['Unnamed: 0', 'text', 'emotion']]
splits = [*datasets.keys()]

print("Data columns:", cols)
print("Emotions columns:", emotion_cols)

Data columns: ['text', 'emotion', 'marah', 'jijik', 'takut', 'senang', 'sedih', 'terkejut', 'biasa']
Emotions columns: ['marah', 'jijik', 'takut', 'senang', 'sedih', 'terkejut', 'biasa']


In [9]:
class2id = {class_:id for id, class_ in enumerate(emotion_cols)}
id2class = {id:class_ for class_, id in class2id.items()}

print("Class to ID:\n")
pprint(class2id, width=1)
print()
print("ID to Class:\n")
pprint(id2class, width=1)

Class to ID:

{'biasa': 6,
 'jijik': 1,
 'marah': 0,
 'sedih': 4,
 'senang': 3,
 'takut': 2,
 'terkejut': 5}

ID to Class:

{0: 'marah',
 1: 'jijik',
 2: 'takut',
 3: 'senang',
 4: 'sedih',
 5: 'terkejut',
 6: 'biasa'}


## Preprocess Data

In [11]:
tokenizer = AutoTokenizer.from_pretrained(hf_model_id)

In [14]:
def one_hot_encode_emotion(emotion, emotion_cols):
    emotions = emotion.replace(" ", "").split(",")
    one_hot_emotion = [1.0 if emotion_col in emotions else 0.0 for emotion_col in emotion_cols] # Ensure that the label is float, not int
    return one_hot_emotion

def preprocess_function(data):
   text = data['text']
   emotion = data['emotion']
   labels = one_hot_encode_emotion(emotion, emotion_cols)
   data = tokenizer(text, truncation=True)
   data['labels'] = labels
   return data

tokenized_datasets = {split: datasets[split].map(preprocess_function) for split in splits}

In [22]:
# Sanity check
data = tokenized_datasets['train'][5]

print("Text:", data['text'])
print("Emotion(s):", data['emotion'])
print("Labels:", data['labels'], '-->', emotion_cols)

Text: ☝️Cung urang Kuningan, Pas urang waduk Darma, Kpan atuh main ka waduk Darma Dai, ??? Di antos, Aa sareng tteh Na, 🤗🤗🤗
Emotion(s): senang
Labels: [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0] --> ['marah', 'jijik', 'takut', 'senang', 'sedih', 'terkejut', 'biasa']


In [16]:
data_collator = DataCollatorWithPadding(tokenizer)

# Model

In [17]:
model = AutoModelForSequenceClassification.from_pretrained(
    hf_model_id, 
    # num_labels=len(emotion_cols),
    # id2label=id2class, label2id=class2id,
    problem_type = "multi_label_classification",
)
print(model.classifier)

config.json:   0%|          | 0.00/863 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

Linear(in_features=768, out_features=4, bias=True)


# Evaluation

In [27]:
def sigmoid(x):
   return 1/(1 + np.exp(-x))

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    probs = sigmoid(predictions)
    y_pred = (probs > 0.5).astype(int)
    y_true = labels.astype(int)

    # Compute F1 score for each type of averaging method
    f1_micro = f1_score(y_true, y_pred, average='micro', zero_division=0.0)
    f1_macro = f1_score(y_true, y_pred, average='macro', zero_division=0.0)
    # f1_weighted = f1_score(y_true, y_pred, average='weighted', zero_division=0.0)
    # f1_samples = f1_score(y_true, y_pred, average='samples', zero_division=0.0)
    f1_labels = f1_score(y_true, y_pred, average=None, zero_division=0.0)
    f1_labels_dict = {f'f1_label_{emotion_cols[i]}': f1_labels[i] for i in range(len(f1_labels))}

    return {
        'f1_micro': f1_micro,
        'f1_macro': f1_macro,
        **f1_labels_dict,
    }

train_args = TrainingArguments(
    # Training config
    per_device_train_batch_size=2,
    # num_train_epochs=3,
    num_train_epochs=1,
    learning_rate=2e-5,
    weight_decay=0.01,

    # Logging config for training
    logging_strategy='steps',
    logging_steps=100,

    # Evaluation config during training
    per_device_eval_batch_size=2,
    eval_strategy='steps',
    eval_steps=100,

    # Model saving config
    # output_dir=project_name,
    output_dir=hf_model_id,
    save_strategy='epoch',
    # load_best_model_at_end=True,
)

trainer = Trainer(
    model=model,
    args=train_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['val'],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

<ipython-input-27-149fa949f8da>:48: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [31]:
eval = trainer.evaluate(eval_dataset=tokenized_datasets['test'])

import pandas as pd
pd.DataFrame(eval, index=[0])

ValueError: Target size (torch.Size([2, 7])) must be the same as input size (torch.Size([2, 4]))